In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv(
    "./data/0_inputData/assembly_summary_genbank_Uni_SeqSpeHttp.txt", sep="\t"
)

In [3]:
pairDict = dict()
with open(
    "./data/0_inputData/assembly_summary_genbank_Uni_SeqSpeHttp_taxion.txt", "r"
) as f:
    for line in f:
        lines = line.strip().split("\t")
        pairDict[lines[-1]] = [i for i in lines[-2::-1]]

In [ ]:
def ifGenus(genus, organism_name):
    for i in genus:
        if " " in i or i[0].islower() or i not in organism_name:
            continue
        return i

In [5]:
for index, row in df.iterrows():
    organism_name = row["organism_name"]
    try:
        df.at[index, "genus"] = ifGenus(pairDict[organism_name], organism_name)
        df.at[index, "Kingdom"] = pairDict[organism_name][-1]
    except:
        pass

In [6]:
df.to_csv(
    "./data/0_inputData/assembly_summary_genbank_Uni_SeqSpeHttp_R.txt", index=False
)

In [7]:
df["fileName"] = df["ftp_path"].str.split("/").str[-1] + "_" + "cds_from_genomic.fna.gz"

In [ ]:
df = df.dropna()
df = df[~df["Kingdom"].isin(["Viruses", "unclassified entries", "other entries"])]
df = df[
    ~df["genus"].str.contains(
        "unclassified|Candidatus|samples|complex|group|unspecified|cluster|uncultured",
        na=False,
    )
]
df = df[
    ~df["organism_name"].str.contains(
        "unclassified|Candidatus|samples|complex|group|unspecified|cluster|uncultured",
        na=False,
    )
]

In [ ]:
import os

for index, value in tqdm(df["fileName"].items()):
    file_path = "./data/4_filterSeq/" + value[:-3]
    if not os.path.exists(file_path):
        df.drop(index, inplace=True)

77293it [19:33, 65.88it/s] 


In [10]:
df_grouped_misc = df.groupby("genus").size().reset_index(name="genusCount")
df_grouped_misc = df_grouped_misc.sort_values(by="genusCount", ascending=False)
df_grouped_misc.to_csv("./misc/countNum.csv", index=False)

In [11]:
df_grouped_misc = df_grouped_misc[df_grouped_misc["genusCount"] >= 5]

In [12]:
df_grouped = pd.merge(df, df_grouped_misc, on="genus", how="right")

In [ ]:
df_grouped["genus"] = df_grouped["genus"].str.replace("/", "_")
df_grouped["genus"] = df_grouped["genus"].str.replace(" ", "_")
df_grouped["genus"] = df_grouped["genus"].str.replace("\t", "_")
df_grouped["genus"] = df_grouped["genus"].str.replace("(", "_")
df_grouped["genus"] = df_grouped["genus"].str.replace(")", "_")
df_grouped["genus"] = df_grouped["genus"].str.replace(".", "_")

In [14]:
df_grouped.to_pickle("./misc/df_grouped.pkl")